In [9]:
import cv2
import tensorflow as tf

# Load the pre-trained CNN model
model_path = "your/model/path/pet_person_model.h5"
model = tf.keras.models.load_model(model_path)

# Set up the camera feed
camera = cv2.VideoCapture(0)  # Use the default camera

# Define the motion detection parameters
delta_thresh = 5
min_area = 500

# Initialize the first frame
_, frame = camera.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (21, 21), 0)
first_frame = gray

# Start the motion tracking loop
while True:
    # Read the next frame from the camera
    _, frame = camera.read()

    # Convert the frame to grayscale and apply blur
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Calculate the absolute difference between the current frame and the first frame
    frame_delta = cv2.absdiff(first_frame, gray)
    thresh = cv2.threshold(frame_delta, delta_thresh, 255, cv2.THRESH_BINARY)[1]

    # Dilate the thresholded image to fill in holes
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through each contour and check if it matches the person or pet class
    for contour in contours:
        if cv2.contourArea(contour) < min_area:
            continue

        # Extract the bounding box coordinates for the contour
        (x, y, w, h) = cv2.boundingRect(contour)

        # Extract the region of interest (ROI) from the original color image
        roi = frame[y:y+h, x:x+w]

        # Preprocess the ROI and classify it using the CNN model
        roi = cv2.resize(roi, (150, 150))
        roi = tf.keras.applications.mobilenet_v2.preprocess_input(roi)
        roi = tf.expand_dims(roi, axis=0)
        predictions = model.predict(roi)

        # Determine if the ROI matches the person or pet class
        if predictions[0][0] > 0.5:
            label = "Person"
            color = (0, 255, 0)  # Green
        else:
            label = "Pet"
            color = (0, 0, 255)  # Red

        # Draw the bounding box and label on the original color image
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting image
    cv2.imshow("Security Feed", frame)

    # Wait for a keypress or until the user quits the program
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# Clean up the camera and close all windows
camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 17ms/step
